# Import libraries

In [1]:
import os
import sys
import glob

import matplotlib.pyplot as plt
import cv2
import numpy as np
import importlib
from process_frames import Processor

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import pandas as pd
    from tqdm import tqdm
    tqdm.pandas()
from IPython.display import display

# Choose options
If you want to process image, set PROCESS_IMAGES to True
If you want to process data, set PROCESS_DATA to True

In [2]:
PROCESS_IMAGES = True
PROCESS_DATA = True

# List images

In [3]:
path = os.getcwd()
os.chdir(path)
sys.path.append(path)
sys.path.append(os.path.join(path, 'scripts'))

path_color = '../data/color/'
path_depth = '../data/depth/'
path_mosaic = '../data/mosaic/'

#Create mosaic path if it not exits
if (not os.path.isdir(path_mosaic)):
    os.mkdir(path_mosaic)

file_list = glob.glob(path_depth+"*")
file_list = [os.path.basename(filename) for filename in file_list]

print(len(file_list),"images found")

2069 images found


# Generate data

In [4]:
if PROCESS_IMAGES:
    processor = Processor()
    areas = {filename: processor.process_image(depth_filename=os.path.join(path_depth, filename),
                          color_filename=os.path.join(path_color, filename),
                          mosaic_path=path_mosaic) for filename in file_list}
    # Prepare data
    for area in areas:    
        if areas[area] is None:
            areas[area] = ['','','','','','','','','','','','','','','']    
    areas_dataframe = pd.DataFrame.from_dict(areas, orient='index',
         columns=['area', 'x', 'y', 'width', 'height', 'MA', 'ma', 'centroid_x', 'centroid_y', 'orientation', '% area', 'center distance', 'excentricity', 'perimeter', 'curvature_coef'])
    areas_dataframe.index.name = 'uuid'
    areas_dataframe.index = areas_dataframe.index.str.replace('.npy', '',)
    print("Generated data")
    display(areas_dataframe.head())
    # Read csv file with weights
    weight_data = pd.read_csv(os.path.join("..", 'data', 'id_weight_sex_uuid.csv'), usecols=['uuid', 'id', 'sex', 'weight'])
    print("Loaded data")
    display(weight_data.head())
    # Save calculated information in csv file
    mix = pd.merge(weight_data, areas_dataframe, on='uuid', how='left')
    mix.to_csv(os.path.join("..", 'data', 'id_weight_sex_uuid_data.csv'))
    print("Final data")
    display(mix.head())

Generated data


,area,x,y,width,height,MA,ma,centroid_x,centroid_y,orientation,% area,center distance,excentricity,perimeter,curvature_coef
uuid,,,,,,,,,,,,,,,
0018b76693a14e20bce3027ff67dc055,273025,196,119,576,346,304.147,563.124,258.378,474.895,-1.34823,50.0156,54.1115,1.64309,1727.47,0.733157
001a83383a7e43e994dbf5b1796f97cc,,,,,,,,,,,,,,,
00607e166c6e4054ab75451aacfbe32d,265084,186,118,592,339,307.1,574.335,257.151,476.759,-1.40705,51.4239,55.4765,1.37831,1757.66,0.741016
00635ae09a834ebf86ee85c58c12ff82,201017,109,108,565,245,250.569,523.747,235.903,407.501,-1.55165,49.9597,17.0004,0.510114,1552.63,0.666081
0066778f47734ca7a71be790f562c551,271063,195,120,578,344,306.357,563.429,259.034,475.183,-1.36629,50.4843,54.6081,1.57173,1733.13,0.736683


Loaded data


,uuid,id,sex,weight
0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5
1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1
2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5
3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5
4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8


Final data


,uuid,id,sex,weight,area,x,y,width,height,MA,ma,centroid_x,centroid_y,orientation,% area,center distance,excentricity,perimeter,curvature_coef
0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5,292877,103,105,564,312,226.871,589.829,291.267,390.795,-1.3058,46.2195,61.0808,0.838312,1797.36,0.928812
1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1,308998,80,202,648,256,217.186,681.105,328.945,427,1.44142,58.5333,88.9953,1.37597,1741.82,0.969556
2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5,292175,195,103,569,324,167.003,671.221,270.915,489.139,1.18351,36.0346,72.1032,0.864987,1568.81,0.853786
3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5,,,,,,,,,,,,,,,
4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8,274669,117,181,613,210,188.983,637.33,288.286,444.696,1.52045,60.0394,52.534,0.751995,1589.51,0.97712


# Process data

In [5]:
if PROCESS_DATA:
    file_data = pd.read_csv(os.path.join("..", 'data', 'id_weight_sex_uuid_data.csv'), usecols=['uuid', 'id', 'sex', 'weight', 'area', 'x', 'y', 'width', 'height', 'MA', 'ma', 'centroid_x', 'centroid_y', 'orientation', '% area', 'center distance', 'excentricity', 'perimeter', 'curvature_coef'])
    print("Load final data")
    display(file_data.head())

Load final data


,uuid,id,sex,weight,area,x,y,width,height,MA,ma,centroid_x,centroid_y,orientation,% area,center distance,excentricity,perimeter,curvature_coef
0,3b144fdfdb9f4e02b0f65aed674fbd72,0268,1,24.5,292877.074890,103.0,105.0,564.0,312.0,226.871475,589.829285,291.266698,390.794758,-1.305798,46.219501,61.080786,0.838312,1797.356550,0.928812
1,960fd92c91f541f6b85889426c39a0fe,0261,1,27.1,308998.472633,80.0,202.0,648.0,256.0,217.185989,681.104919,328.944662,427.000459,1.441417,58.533350,88.995256,1.375974,1741.817446,0.969556
2,ca10f050724d47f5971c3530ef341a8d,0236,0,25.5,292174.501116,195.0,103.0,569.0,324.0,167.003387,671.221436,270.915334,489.139166,1.183508,36.034621,72.103182,0.864987,1568.812394,0.853786
3,3c4dcc3c6d83425dab0d0acd75227f99,0236,0,25.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46d3f67bbf684992ac945ce7c1991ae7,0238,1,25.8,274668.662187,117.0,181.0,613.0,210.0,188.982895,637.330017,288.285815,444.695562,1.520453,60.039439,52.534048,0.751995,1589.508790,0.977120
